## What is this?
Now we have some utilities we used often. It's time to make it more useful.

In [27]:
import sys
import os
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.utils.visualize_util import plot
from matplotlib.pyplot import show, plot
import pandas as pd
import numpy as np
import math
import random
random.seed(0)

model_name = "test_keras"

def weights_dir(model_version):
    return "{0}/Desktop/{1}-{2}".format(os.environ['HOME'], model_name, model_version)

def best_weights_path(model_version):
    files = os.listdir(weights_dir(model_version))
    files.sort()
    return "{0}/{1}".format(weights_dir, files[0])

def default_callback_list(model_version):
    os.makedirs(weights_dir(model_version), exist_ok=True)
    filepath = weights_dir(model_version) + "/{loss:.4f}"
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
    return [checkpoint]

    
print(weights_dir(1))
#print(best_model_path(1))
print(default_callback_list(1))


Using TensorFlow backend.


/Users/higepon/Desktop/test_keras-1
